In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import ray
from ray import tune
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.schedulers import HyperBandScheduler
from sklearn.preprocessing import LabelBinarizer
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import psutil
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [3]:
complete_df = pd.read_csv("complete_dataset.csv")

In [4]:
complete_df_re = complete_df.iloc[:, np.r_[1:12, 17:21]]
complete_df_re = complete_df_re.dropna()
complete_df_re.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,sub-emotion,emotion
0,0.722,0.7190,6.0,-8.565,0.0,0.0540,0.02220,0.223000,0.1100,0.955,148.039,187239.0,3.0,Happy,Wonder
1,0.378,0.3150,7.0,-6.743,1.0,0.0304,0.80200,0.000000,0.5050,0.521,74.663,48000.0,4.0,Happy,Wonder
2,0.291,0.0462,5.0,-22.589,1.0,0.0345,0.92500,0.000000,0.1060,0.276,171.368,152773.0,4.0,Happy,Wonder
3,0.722,0.7970,0.0,-5.179,1.0,0.0887,0.00448,0.339000,0.0989,0.273,122.025,177523.0,4.0,Happy,Wonder
4,0.651,0.7860,9.0,-3.725,1.0,0.0761,0.00350,0.000070,0.1790,0.414,109.019,178421.0,4.0,Happy,Wonder
5,0.774,0.7840,10.0,-4.549,1.0,0.0351,0.19400,0.000011,0.2930,0.533,127.042,177689.0,4.0,Happy,Wonder
6,0.498,0.7450,11.0,-5.801,0.0,0.0316,0.05740,0.000000,0.2550,0.258,125.973,177143.0,4.0,Happy,Wonder
7,0.751,0.7840,8.0,-4.964,1.0,0.1380,0.05210,0.000026,0.0629,0.533,106.069,164151.0,4.0,Happy,Wonder
8,0.728,0.9470,0.0,-4.220,0.0,0.0828,0.01770,0.002540,0.0987,0.553,125.874,238265.0,4.0,Happy,Wonder
9,0.537,0.9640,5.0,-2.677,0.0,0.0552,0.01800,0.001380,0.1600,0.509,125.992,208254.0,4.0,Happy,Wonder


In [5]:
encoder = LabelEncoder()
encoder.fit(complete_df_re.iloc[:, -1])
encoded_Y = encoder.transform(complete_df_re.iloc[:, -1])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
X_train, X_test, y_train, y_test = train_test_split(complete_df_re.iloc[:, :-2], dummy_y)

In [6]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
def train(config, X_tunetrain=None, y_tunetrain=None, X_tunetest=None, y_tunetest=None):
    model = Sequential()
    model.add(layer=keras.layers.Input(shape=X_train.shape[1],))
    for i in range(config['layers']):
        model.add(Dense(config["units"], activation='relu'))
        model.add(Dropout(config['dropout']))
        if config['batchnorm'] == "True": 
            model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(9, activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=config['lr'], decay=1e-6), metrics=['accuracy'])
    model.fit(X_tunetrain, y_tunetrain, batch_size=config['batch_size'], validation_data=(X_tunetest, y_tunetest), epochs=25, callbacks=[TuneReportCallback({"mean_accuracy": "accuracy"})])

In [8]:
config = {
        "batch_size" : tune.grid_search([64,128,256, 512]),
        "layers":     tune.grid_search([i for i in range(1,150,3)]),     # maximum number of layers
        "units":      tune.grid_search([128, 256, 512, 1024]),
        "dropout":    tune.uniform(0, 1),
        "lr":          tune.uniform(0.0001, 0.1),
        "batchnorm" : tune.choice(["True", "False"])
    }

In [9]:
hyperband_scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='mean_accuracy',
    mode='max',
    max_t=10,
    reduction_factor=4)

In [10]:
ray.init(num_gpus=1)
print(ray.get_gpu_ids())
# prints []

2022-05-16 00:38:35,361	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265
2022-05-16 00:38:36,544	WARNING worker.py:523 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.


[]


In [11]:
analysis = tune.run(
    tune.with_parameters(train, X_tunetrain=X_train, y_tunetrain=y_train, X_tunetest=X_test, y_tunetest=y_test),
    name="exp",
    config=config,
    verbose=1,
    resources_per_trial={
            "gpu": 0,
            "cpu": 2
        },scheduler=hyperband_scheduler)

2356/2627 [=========================>....] - ETA: 13s - loss: 2.2351 - accuracy: 0.1318
